In [12]:
# Loaded variable 'df' from URI: /root/2024_p2_mcq_plus_sample1000.tsv
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
df = pd.read_csv(r'/root/2024_p2_mcq_plus_mllm.tsv', sep='\t')
print(len(df))
# answer 不在ABCD中
df = df[df['answer'].isin(['A', 'B', 'C', 'D']) == True]
print(len(df))

9677
9633


In [14]:
def extract_arxiv_category(html_text):
    # 正则表达式模式，用于匹配 arXiv ID
    soup = BeautifulSoup(html_text, 'html.parser')
    primary_subject = soup.find('span', class_='primary-subject').text  
    return primary_subject
def get_category(id):
    url = f'https://arxiv.org/abs/{id}'
    
    response = requests.get(url)
    response.raise_for_status()
    time.sleep(1)
    category = extract_arxiv_category(response.content.decode())
    print(category)
    return category
    

In [17]:
df['paper_ids'] = df['image_path'].apply(lambda x: x.split('/')[-1].split('_')[0])
print(df)
# 按paper_ids分组
grouped = df.groupby('paper_ids')

      index                                         image_path  \
0         1  /root/benchmark/data/2024/2309.09771v2_figatla...   
1         2  /root/benchmark/data/2024/2309.09771v2_figflav...   
2         3  /root/benchmark/data/2024/2309.09771v2_figener...   
3         4  /root/benchmark/data/2024/2309.16560_figselfca...   
4         5  /root/benchmark/data/2024/2309.16560_Figrmscon...   
...     ...                                                ...   
9672   9673  /root/benchmark/data/2024/2407.17577_secQindex...   
9673   9674  /root/benchmark/data/2024/2407.17613_figflags.jpg   
9674   9675  /root/benchmark/data/2024/2407.17613_figrepeat...   
9675   9676  /root/benchmark/data/2024/2407.17613_figjkabsm...   
9676   9677    /root/benchmark/data/2024/2407.18912_figsmf.jpg   

                                               question  \
0     Which of the following celestial objects is al...   
1     What does the solid blue line in the flavor tr...   
2     What type of supernova i

In [18]:
# 每组应用 get_category，并将结果存储在新的列中
df['category'] = grouped['paper_ids'].apply(lambda x: get_category(x.iloc[0]))

High Energy Physics - Phenomenology (hep-ph)
Astrophysics of Galaxies (astro-ph.GA)
Solar and Stellar Astrophysics (astro-ph.SR)
Solar and Stellar Astrophysics (astro-ph.SR)
Instrumentation and Methods for Astrophysics (astro-ph.IM)
Cosmology and Nongalactic Astrophysics (astro-ph.CO)
Cosmology and Nongalactic Astrophysics (astro-ph.CO)
Astrophysics of Galaxies (astro-ph.GA)
High Energy Astrophysical Phenomena (astro-ph.HE)
High Energy Astrophysical Phenomena (astro-ph.HE)
Astrophysics of Galaxies (astro-ph.GA)
Earth and Planetary Astrophysics (astro-ph.EP)
Astrophysics of Galaxies (astro-ph.GA)
Astrophysics of Galaxies (astro-ph.GA)
Astrophysics of Galaxies (astro-ph.GA)
Astrophysics of Galaxies (astro-ph.GA)
Cosmology and Nongalactic Astrophysics (astro-ph.CO)
Cosmology and Nongalactic Astrophysics (astro-ph.CO)
Earth and Planetary Astrophysics (astro-ph.EP)
Cosmology and Nongalactic Astrophysics (astro-ph.CO)
Cosmology and Nongalactic Astrophysics (astro-ph.CO)
Instrumentation and M

In [28]:
category = [i.strip() for i in open('/root/category.txt', 'r').readlines()]
assert len(category) == len(grouped)


In [30]:

# 创建一个字典，将分组的键映射到类别
group_keys = list(grouped.groups.keys())
category_dict = dict(zip(group_keys, category))

# 使用 transform 方法将类别分配给每组的所有成员
df['category'] = df['paper_ids'].transform(lambda x: category_dict[x])

In [32]:
df.rename(columns={'explian': 'explain'}, inplace=True)
df.rename(columns={'sum_score': 'mllm_sum'}, inplace=True)
df.head()


,index,image_path,question,A,B,C,D,answer,explain,InternVL2-26B_prediction,...,llava_score,InternVL2-40B_prediction,InternVL2-40B_score,glm-4v-9b_prediction,glm-4v-9b_score,InternVL2-76B_prediction,InternVL2-76B_score,mllm_sum,paper_ids,category
0,1,/root/benchmark/data/2024/2309.09771v2_figatla...,Which of the following celestial objects is al...,Betelgeuse,Eta Carinae,Canopus,Ursa Minor,B,"According to the skymap, objects with a declin...",C,...,0,D,0,D,0,D,0,1,2309.09771v2,High Energy Physics - Phenomenology (hep-ph)
1,2,/root/benchmark/data/2024/2309.09771v2_figflav...,What does the solid blue line in the flavor tr...,The 1σ confidence interval for the analysis us...,The 3σ confidence interval for the analysis us...,The 95% confidence interval from IceCube using...,The expected flavor ratios for neutrinos from ...,A,The solid blue line in the flavor triangle rep...,A,...,0,C,0,C,0,C,0,3,2309.09771v2,High Energy Physics - Phenomenology (hep-ph)
2,3,/root/benchmark/data/2024/2309.09771v2_figener...,What type of supernova is represented by the e...,Type Ia,Type II,Type IIn,Type Ib,C,"The image is labeled as ""Type IIn, \(D_* = 1.0...",C,...,1,C,1,C,1,C,1,8,2309.09771v2,High Energy Physics - Phenomenology (hep-ph)
3,4,/root/benchmark/data/2024/2309.16560_figselfca...,Which candidate exhibits the lowest dynamic ra...,Candidate 1,Candidate 2,Candidate 3,Candidate 4,B,The second candidate (Candidate 2) shows the l...,D,...,0,D,0,B,1,C,0,1,2309.16560,Instrumentation and Methods for Astrophysics (...
4,5,/root/benchmark/data/2024/2309.16560_Figrmscon...,What is the primary reason for the increase in...,Residual direction-dependent effects (DDEs),Primary beam correction,Source density variation,Instrumental sensitivity limits,B,The increase in RMS noise level from the cente...,A,...,0,D,0,C,0,A,0,0,2309.16560,Instrumentation and Methods for Astrophysics (...


In [33]:
df.to_csv(r'/root/2024_p2_mcq_plus_mllm.tsv', sep='\t', index=False)

In [36]:
df = pd.read_csv(r'/root/2024_p2_mcq_plus_mllm.tsv', sep='\t')
# 将最后两个字段放在前面
df = df[df.columns[:2].tolist() + df.columns[-2:].tolist()+ df.columns[2:-2].tolist()]
df.head()


,index,image_path,paper_ids,category,question,A,B,C,D,answer,...,InternVL2-8B_score,llava_prediction,llava_score,InternVL2-40B_prediction,InternVL2-40B_score,glm-4v-9b_prediction,glm-4v-9b_score,InternVL2-76B_prediction,InternVL2-76B_score,mllm_sum
0,1,/root/benchmark/data/2024/2309.09771v2_figatla...,2309.09771v2,High Energy Physics - Phenomenology (hep-ph),Which of the following celestial objects is al...,Betelgeuse,Eta Carinae,Canopus,Ursa Minor,B,...,0,D,0,D,0,D,0,D,0,1
1,2,/root/benchmark/data/2024/2309.09771v2_figflav...,2309.09771v2,High Energy Physics - Phenomenology (hep-ph),What does the solid blue line in the flavor tr...,The 1σ confidence interval for the analysis us...,The 3σ confidence interval for the analysis us...,The 95% confidence interval from IceCube using...,The expected flavor ratios for neutrinos from ...,A,...,0,C,0,C,0,C,0,C,0,3
2,3,/root/benchmark/data/2024/2309.09771v2_figener...,2309.09771v2,High Energy Physics - Phenomenology (hep-ph),What type of supernova is represented by the e...,Type Ia,Type II,Type IIn,Type Ib,C,...,1,C,1,C,1,C,1,C,1,8
3,4,/root/benchmark/data/2024/2309.16560_figselfca...,2309.16560,Instrumentation and Methods for Astrophysics (...,Which candidate exhibits the lowest dynamic ra...,Candidate 1,Candidate 2,Candidate 3,Candidate 4,B,...,0,C,0,D,0,B,1,C,0,1
4,5,/root/benchmark/data/2024/2309.16560_Figrmscon...,2309.16560,Instrumentation and Methods for Astrophysics (...,What is the primary reason for the increase in...,Residual direction-dependent effects (DDEs),Primary beam correction,Source density variation,Instrumental sensitivity limits,B,...,0,A,0,D,0,C,0,A,0,0


In [37]:
df.to_csv(r'/root/2024_p2_mcq_plus_mllm.tsv', sep='\t', index=False)